# Read in Data

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import psycopg2 as pg
import io
import pandas as pd

## Read in login and pass
with open('rootkey.csv') as f:
    login = f.readline().strip('\n')
    passw = f.readline().strip('\n')

panama = pd.read_csv('panama/panama_papers_cleaned.csv', sep='|')
paradise = pd.read_csv('paradise/paradies_papers_cleaned.csv', sep='|')

/home/ec2-user/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Concat

In [2]:
panama['type'] = 'panama'
paradise['type'] = 'paradise'
merged = pd.concat([panama, paradise])
merged['name_entity'] = merged['name_entity'].str.upper()
merged.to_csv('panama_and_paradise_merged.csv', sep='|', na_rep='nan')
merged.head()

,name_officer,name_entity,countries_officer,address_officer,countries_entity,address_entity,type
0,KIM SOO IN,TOPSON MARK LIMITED,South Korea,MR WALTER KIM; ESQ. \t CO RODI & POLLOCK 444 ...,United States,MR WALTER KIM; ESQ. \t CO RODI & POLLOCK 444 ...,panama
1,Tian Yuan,CELLO HOLDINGS LIMITED,China,"RAPID MANAGEMENT SERVICES COMPANY 4TH FLOOR, W...",Hong Kong,"RAPID MANAGEMENT SERVICES COMPANY 4TH FLOOR, W...",panama
2,GREGORY JOHN SOLOMON,HANUMAN ENERGY LTD.,Australia,105 SOPHIA ROAD #01-03 SINGAPORE 228170,Singapore,105 SOPHIA ROAD #01-03 SINGAPORE 228170,panama
3,MATSUDA MASUMI,"BAOJ INTERNATIONAL CO., LTD.",Japan,ORION HOUSE SERVICES (HK) LIMITED ROOM 1401; 1...,Hong Kong,ORION HOUSE SERVICES (HK) LIMITED ROOM 1401; 1...,panama
4,HO THUY NGA,BLUEWATER ALLIANCE LIMITED,Viet Nam,CORPORATE MANAGEMENT SERVICES LIMITED 1301 BAN...,Hong Kong,CORPORATE MANAGEMENT SERVICES LIMITED 1301 BAN...,panama


## Connect to Database and Upload

In [8]:
engine_string = 'postgresql+psycopg2://' + login + ':' + passw + '@offshoredb.cwwa5jzhqwm0.us-east-1.rds.amazonaws.com:5432/offshoredb'
engine = create_engine(engine_string)
conn=engine.raw_connection()
cur = conn.cursor()

In [10]:
sql_command = '''CREATE TABLE offshore_papers (
    index int,
    name_officer varchar,
    name_entity varchar,
    country_officer varchar,
    address_officer varchar,
    countries_entity varchar,
    address_entity varchar
    type varchar
    )'''

cur.execute(sql_command)
conn.commit()

In [12]:
def upload_table(table_path, table_name):
    with open(table_path) as f:
        next(f) ## Skip header
        cur.copy_from(f, table_name, sep='|')

    conn.commit()
    conn.close()

In [13]:
upload_table('panama_and_paradise_merged.csv', 'offshore_papers')